# Today I will Conquer ESCIP

## and I will also go grocery shopping

In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Dyanmics of particle in a box with a delta potential\n",
    "[Jay Foley and Lane Tolley, University of North Carolina Charlotte](https://foleylab.github.io/)\n",
    "\n",
    "#### Objectives\n",
    "- To demonstrate the use of the numerical methods, including the Euler method and the Runge-Kutta method, to solve the time-dependent Schrodinger equation\n",
    "- To illustrate the dynamics of a particle in a box with a delta potential\n",
    "\n",
    "#### Learning Outcomes\n",
    "By the end of this workbook, students should be able to\n",
    "- Express the time-dependent Schrodinger equation using matrices for the Hamiltonian operator and vectors for the wavefunction\n",
    "- Utilize matrix-vector operations in `numpy` to compte time-derivatives of the wavefunction\n",
    "- Utilize numerical integration methods, particularly the Euler method and the Runge-Kutta method, to evolve the wavefunction in time.\n",
    "\n",
    "\n",
    "#### Summary\n",
    "We will investigate the particle in a box with a delta potential that was explored in the notebook on the [Linear Variational Method](https://escip.io/notebooks/linear_variational_method.html), but in the context of the time-dependence induced by the delta potential on an initial state, rather than through the application of the variational method to identify stationary states of this Hamiltonian.  We will choose an initial state to be the ground state of the ordinary particle in a box system, so again we will find it useful to build the Hamiltonian matrix in the basis of\n",
    "the particle in a box states, and to express the wavefunction in terms of the amplitudes of each of these basis states.\n",
    "\n",
    "\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Step 0:  Import libraries and initialize a plot object for the animation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "\n",
    "from matplotlib import animation, rc\n",
    "from IPython.display import HTML\n",
    "\n",
    "\n",
    "# First set up the figure, the axis, and the plot element we want to animate\n",
    "fig, ax = plt.subplots()\n",
    "plt.close()\n",
    "\n",
    "### parameters for the box size and delta function\n",
    "x0 = 5\n",
    "L = 10\n",
    "Amp = np.sqrt(2/L)\n",
    "\n",
    "### grid of x values for the functions\n",
    "x = np.linspace(0, L, 500)\n",
    "\n",
    "### parameters for plot\n",
    "ax.set_xlim((0, L))\n",
    "ax.set_ylim((-2*Amp, 2*Amp))\n",
    "\n",
    "line, = ax.plot([], [], lw=2)\n",
    "\n",
    "# initialization function: plot the background of each frame\n",
    "def init():\n",
    "    line.set_data([], [])\n",
    "    return (line,)\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Approach\n",
    "In general, the dynamics of a quantum system are determined by the time-dependent Schrodinger equation (TDSE),\n",
    "$$ i\\hbar \\frac{d}{dt}\\Psi(x, t) = \\hat{H} \\Psi(x, t). \\tag{1} $$\n",
    "\n",
    "Just as [before](https://escip.io/notebooks/linear_variational_method.html) can write the Hamiltonian operator for this system between $x=0$ and $x=10$ as follows:\n",
    "\n",
    "$$\\hat{H} = -\\frac{\\hbar^2}{2m} \\frac{d^2}{dx^2} + \\delta(x-5).  \\tag{2} $$\n",
    "\n",
    "Also as before, we can build a matrix representation of this Hamiltonian $ {\\bf H}$  by introducing a basis.  We will use the \n",
    "energy eigenfunctions of the ordinary particle in a box as this basis, where these basis functions are given by\n",
    "\n",
    "$$ \\psi_n(x) = \\sqrt{ \\frac{2}{10} } {\\rm sin}\\left( \\frac{n \\pi x}{10} \\right), \\tag{3} $$\n",
    "\n",
    "and the elements of $ {\\bf H}$ will be given by \n",
    "\\begin{equation}\n",
    "H_{nm} = \\int_0^{10} \\psi^*_n(x) \\hat{H} \\psi_m(x) dx. \\tag{4} \n",
    "\\end{equation}\n",
    "Because the basis set comprised of all $\\psi_n(x)$ is complete, we can represent our wavefunction at any instant in time as a linear combination of these basis functions:\n",
    "\n",
    "$$ \\Psi(x, t) = \\sum_n c_n(t) \\psi_n(x) \\tag{5} $$\n",
    "where we are denoting that the basis functions are time-independent, and the time-dependence is contained only in\n",
    "the expansion coefficients $c_n(t)$.  Thus we can map the time-dependence of the collection of these coefficients, which we will call ${\\bf c}(t)$ to the time-dependence of the wavefunction.  \n",
    "This suggests that the TDSE can be recast as\n",
    "$$ i\\hbar \\frac{d}{dt} {\\bf c}(t) = {\\bf H} {\\bf c}(t). \\tag{6} $$"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "#### Numerically Solving the TDSE\n",
    "The key idea for numerically solving the TDSE is that if we know the coefficients at one instant in time,\n",
    "${\\bf c}(t_0)$ and wish to find the coefficients at a later time $t_0 + \\Delta t$, we can approximate \n",
    "${\\bf c}(t_0 + \\Delta t)$ using information about the time derivative of the coefficients given by the TDSE.  The simplest approximation is known as the Euler update, and has the form\n",
    "\n",
    "$$ {\\bf c}(t_0 + \\Delta t) = {\\bf c}(t_0) + \\frac{d}{dt} {\\bf c}(t_0) \\Delta t, \\tag{7} $$\n",
    "\n",
    "where from the TDSE, we can see\n",
    "\n",
    "$$ \\frac{d}{dt} {\\bf c}(t_0) = -\\frac{i}{\\hbar} {\\bf H} {\\bf c}(t_0). \\tag{8} $$"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Questions Part 1:\n",
    "\n",
    "1. We will include the helper functions from before that will build the Hamiltonian matrix in the PIB basis. Add a function called `compute_c_dot(H, c)` that will take the Hamiltonian matrix and a given c vector and return the time-derivative of the c vector using the TDSE using Eq. (8).\n",
    "\n",
    "2. Write a function called `euler_update(H, c, dt)` that takes the Hamiltonian matrix `H`, a c vector `c` and a time increment `dt` and returns the c vector at a future time using Eq. (7).  It can call `compute_c_dot(H, c)` to get the time-derivative of `c`.\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def energy_eigenvalue(n, L, m):\n",
    "    \"\"\" Helper function to take the quantum number n of the particle in a box, the length \n",
    "        of the box L, and the mass of the particle m and return the energy eigenvalue in atomic units.\n",
    "        Both the length and mass should be in atomic units.\n",
    "        \n",
    "    Arguments\n",
    "    ---------\n",
    "    n : int\n",
    "        the quantum state of the particle in a box\n",
    "        \n",
    "    L : float\n",
    "        the length of the box in atomic units\n",
    "    m : float\n",
    "        the mass of the particle in atomic units\n",
    "    \"\"\"\n",
    "        \n",
    "    return n ** 2 * np.pi ** 2 / ( 2 * m * L ** 2)\n",
    "\n",
    "def energy_eigenfunction(n, L, x):\n",
    "    \"\"\" Helper function to take the quantum number n of the particle in a box, the length \n",
    "        of the box L, and x-coordinate value(s) (single value or list) and return the corresponding\n",
    "        energy eigenstate value(s) of the ordinary particle in a box\n",
    "        \n",
    "    Arguments\n",
    "    ---------\n",
    "    n : int\n",
    "        the quantum state of the particle in a box      \n",
    "    L : float\n",
    "        the length of the box in atomic units\n",
    "    x : float (or numpy array of floats)\n",
    "        the position variable for the energy eigenstate in atomic units\n",
    "    \"\"\"\n",
    "    return np.sqrt(2 / L) * np.sin(n * np.pi * x / L)\n",
    "\n",
    "def Hamiltonian_matrix_element(n, m):\n",
    "    \"\"\" Helper function to take two indices (n, m) and return the Hamiltonian matrix element H_{n,m}, \n",
    "        which is <n|H|m> in Dirac's Bra-Ket notation.  This will call Kinetic_matrix_element(n, m) and\n",
    "        Potential_matrix_element(n, m).\n",
    "    \n",
    "    Arguments\n",
    "    ---------\n",
    "    n : int\n",
    "        the index corresponding to the bra state\n",
    "    m : int\n",
    "        the index corresponding to the ket state \n",
    "        \n",
    "    Returns\n",
    "    -------\n",
    "    H_mn : float\n",
    "         the matrix element corresponding to <n|H|m> = <n|T|m> + <n|V|m>\n",
    "         \n",
    "    Example\n",
    "    -------\n",
    "    >>> H_12 = Hamiltonian_matrix_element(1,2)\n",
    "    >>> H_33 = Hamiltonian_matrix_element(3,3)\n",
    "    \n",
    "    Notes\n",
    "    -----\n",
    "    if n == m, you need to include kinetic and potential contributions of <n|H|n>.\n",
    "    e.g. <n|H|n> = <n|T|n> + <n|V|n> = <n|E_n|n> + <n|V|n> = E_n<n|n> + <n|V|n> = E_n + <n|V|n>\n",
    "    where E_n is the energy eigenvalue of the ordinary particle in a box state \\psi_n(x)\n",
    "    \n",
    "    if n != m, you need to include only the potential contribution <n|V|m>.\n",
    "    \n",
    "    \"\"\"\n",
    "    H_nm = Kinetic_matrix_element(n, m) + Potential_matrix_element(n, m)\n",
    "    return H_nm\n",
    "\n",
    "def Kinetic_matrix_element(n, m):\n",
    "    \"\"\" Function to take two indices (n, m) and return the Kinetic Energy matrix element T_{n,m}, \n",
    "        which is <n|T|m> in Dirac's Bra-Ket notation.  \n",
    "    \n",
    "    Arguments\n",
    "    ---------\n",
    "    n : int\n",
    "        the index corresponding to the bra state\n",
    "    m : int\n",
    "        the index corresponding to the ket state \n",
    "        \n",
    "    Returns\n",
    "    -------\n",
    "    T_nm : float\n",
    "         the matrix element corresponding to <n|T|m> = E_m <n|m>\n",
    "         \n",
    "    Example\n",
    "    -------\n",
    "    >>> T_12 = Kinetic_matrix_element(1,2) -> 0\n",
    "    >>> T_33 = Kinetic_matrix_element(3,3) -> E_3\n",
    "    \n",
    "    \"\"\"\n",
    "    T_nm = energy_eigenvalue(m, 10, 1) * (n == m)\n",
    "    return T_nm\n",
    "\n",
    "def Potential_matrix_element(n, m):\n",
    "    \"\"\" Function to take two indices (n, m) and return the Hamiltonian matrix element V_{n,m}, \n",
    "        which is <n|V|m> in Dirac's Bra-Ket notation and V = \\delta(x-5)\n",
    "    \n",
    "    Arguments\n",
    "    ---------\n",
    "    n : int\n",
    "        the index corresponding to the bra state\n",
    "    m : int\n",
    "        the index corresponding to the ket state \n",
    "        \n",
    "    Returns\n",
    "    -------\n",
    "    V_nm : float\n",
    "         the matrix element corresponding to <n|V|m> = \\psi_n(5) * psi_m(5)\n",
    "         \n",
    "    Example\n",
    "    -------\n",
    "    >>> V_12 = Potential_matrix_element(1,2)\n",
    "    >>> V_33 = Potential_matrix_element(3,3)\n",
    "    \"\"\"\n",
    "    V_nm = energy_eigenfunction(n, 10, 5) * energy_eigenfunction(m, 10, 5)\n",
    "    return V_nm\n",
    "\n",
    "def build_matrices(n_basis):\n",
    "    \"\"\" Function that will build and return the Hamiltonian, Kinetic, and Potential energy matrices\n",
    "    \n",
    "    Arguments\n",
    "    ---------\n",
    "    n_basis : int\n",
    "        the number of basis functions used to build the matrices\n",
    "        \n",
    "    Returns\n",
    "    -------\n",
    "    H : n_basis x n_basis array of floats\n",
    "         the total Hamiltonian matrix\n",
    "         \n",
    "    T : n_basis x n_basis array of floats\n",
    "         the kinetic energy matrix\n",
    "         \n",
    "    V : n_basis x n_basis array of floats\n",
    "         the potential energy matrix \n",
    "         \n",
    "    Example\n",
    "    -------\n",
    "    >>> H, T, V = build_matrices(3)\n",
    "    \"\"\"\n",
    "    H = np.zeros((n_basis, n_basis))\n",
    "    T = np.zeros((n_basis, n_basis))\n",
    "    V = np.zeros((n_basis, n_basis))\n",
    "    \n",
    "    for i in range(n_basis):\n",
    "        n = i + 1 # i starts from 0 but n should start at 1\n",
    "        for j in range(n_basis):\n",
    "            m = j + 1 # j starts at 0 but m should start at 1\n",
    "            H[i, j] = Hamiltonian_matrix_element(n, m)\n",
    "            T[i, j] = Kinetic_matrix_element(n, m)\n",
    "            V[i, j] = Potential_matrix_element(n, m)\n",
    "    return H, T, V\n",
    "\n",
    "def compute_c_dot(H, c):\n",
    "    \"\"\" Function that will compute d/dt c(t_0) using -i/hbar H * c \n",
    "    \n",
    "    Arguments\n",
    "    ---------\n",
    "    H : n_basis x n_basis numpy array of floats\n",
    "        the Hamiltonian matrix in atomic units\n",
    "        \n",
    "    c : 1 x n_basis numpy array of complex floats\n",
    "        the vector of coefficients that defines the time-dependent wavefunction\n",
    "    \n",
    "    Returns\n",
    "    -------\n",
    "    c_dot : 1 x n_basis numpy array of complex floats\n",
    "        the vector of time-derivatives of the coefficients\n",
    "    \n",
    "    \"\"\"\n",
    "    # <== define imaginary unit as ci = 0+1j\n",
    "    # <== compute matrix-vector product of H and c to get c_dot\n",
    "    return c_dot\n",
    "\n",
    "def euler_update(H, c, dt):\n",
    "    \"\"\" Function that will take c(t0) and H and return c(t0 + dt)\n",
    "    \n",
    "    Arguments\n",
    "    ---------\n",
    "    H : n_basis x n_basis numpy array of floats\n",
    "        the Hamiltonian matrix in atomic units\n",
    "        \n",
    "    c : 1 x n_basis numpy array of complex floats\n",
    "        the vector of coefficients that defines the time-dependent wavefunction\n",
    "        \n",
    "    dt : float\n",
    "        the increment in time in atomic units\n",
    "    \n",
    "    \n",
    "    Returns\n",
    "    -------\n",
    "    c_new : 1 x n_basis numpy array of complex floats\n",
    "        the vector of coefficients at time t0 + dt\n",
    "    \n",
    "    \"\"\"\n",
    "    # <== call compute_c_dot(H, c) to compute c_dot\n",
    "    # <== compute c_new using c, c_dot, and dt\n",
    "    return c_new\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "The following lines will build the Hamiltonian matrix, define an initial c-vector, and update it using the Euler method for a number of time-steps storing the time-dependent c-vector in a n_basis x n_time numpy array of complex floats"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# number of basis functions\n",
    "n_basis = 50\n",
    "\n",
    "# build H, T, and V\n",
    "Hamiltonian_matrix, Kinetic_matrix, Potential_matrix = build_matrices(n_basis)\n",
    "\n",
    "# number of time steps\n",
    "N_time = 700\n",
    "\n",
    "# create the c_vec array as a n_basis x N_time numpy array of complex floats\n",
    "c_vec = np.zeros((n_basis, N_time), dtype=complex)\n",
    "\n",
    "# define a time increment of 0.005 atomic units\n",
    "dt = 0.001\n",
    "\n",
    "# initialize the c(0) vector as (1+0j, 0, 0, ...) -> initial state = \\psi_1(x)\n",
    "c_vec[0,0] = 1+0j\n",
    "\n",
    "# loop over time-steps and update the c-vector at each time\n",
    "for i in range(1, N_time):\n",
    "    c_vec[:,i] = euler_update(Hamiltonian_matrix, c_vec[:,i-1], dt)\n",
    "\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "The following lines will animate the wavefunction as shown in Eq. (5).  Now, the elements of ${\\bf c}(t)$ are stored\n",
    "in `c_vec`, and the $\\psi_n(x)$ functions can be obtained by calling `energy_eigenfunction(n, L, x)`.  "
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "t = np.zeros(N_time)\n",
    "\n",
    "\n",
    "# animation function. This is called sequentially  \n",
    "def animate(i):\n",
    "    y = np.zeros(len(x),dtype=complex)\n",
    "    for j in range(0, n_basis):\n",
    "        fx = energy_eigenfunction(j+1, 10, x)\n",
    "        y = y + c_vec[j,i] * fx\n",
    "    t[i] = i\n",
    "    line.set_data(x, np.real(y))\n",
    "    return (line,)\n",
    "  \n",
    "\n",
    "anim = animation.FuncAnimation(fig, animate, init_func=init,\n",
    "                             frames=N_time, interval=1, blit=True)\n",
    "\n",
    "# Note: below is the part which makes it work on Colab\n",
    "rc('animation', html='jshtml')\n",
    "anim\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.9.13"
  },
  "vscode": {
   "interpreter": {
    "hash": "2463cbcb379b8a49ed799784d6e0033fd634b3823d327a76d1d60741708bab10"
   }
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}